这一步主要做数据清洗和数据格式化。age和gender给出的数据形式为多值概率，所以选取概率最大的作为用户的基本属性。picnum和pubtime存在一些不符合预期的特征值，需要先做特征清洗。受限于资源限制，所以训练集只保留了两天的数据，而且按照0.01的比例进行了下采样

In [13]:
import pandas as pd
test_info = pd.read_table('../data/test_info.txt',header=None)
print(test_info)

           0           1          2              3  4   5
0          0  2204868548  464495940  1625011774995  2   4
1          1  1623790870  464553715  1625066054894  2   2
2          2  1637010070  464687570  1625053546711  2  10
3          3  1634251474  464661865  1625047014335  2  10
4          4  2352721164  464692079  1625035383911  2  11
...      ...         ...        ...            ... ..  ..
49995  49995  1640585246  464553715  1625036568698  2   1
49996  49996  1987843480  464382231  1625009469794  2   1
49997  49997  2391080328  464617167  1625027566512  5   1
49998  49998   125699342  464172118  1625034692217  2  11
49999  49999  2154437014  464225813  1625008637470  2   3

[50000 rows x 6 columns]


In [14]:
test_info.columns = ['id', 'userid', 'docid', 'timestamp', 'network', 'refresh']
test_info['dt'] = pd.to_datetime(test_info['timestamp'], utc=True,
                               unit='ms').dt.tz_convert('Asia/Shanghai')
test_info['date'] = test_info['dt'].dt.date
test_info['date'] = test_info['date'].astype('str')
print('test data')
print(test_info.head())
# test_info.to_pickle('../data/test.pkl')

test data
   id      userid      docid      timestamp  network  refresh  \
0   0  2204868548  464495940  1625011774995        2        4   
1   1  1623790870  464553715  1625066054894        2        2   
2   2  1637010070  464687570  1625053546711        2       10   
3   3  1634251474  464661865  1625047014335        2       10   
4   4  2352721164  464692079  1625035383911        2       11   

                                dt        date  
0 2021-06-30 08:09:34.995000+08:00  2021-06-30  
1 2021-06-30 23:14:14.894000+08:00  2021-06-30  
2 2021-06-30 19:45:46.711000+08:00  2021-06-30  
3 2021-06-30 17:56:54.335000+08:00  2021-06-30  
4 2021-06-30 14:43:03.911000+08:00  2021-06-30  


In [16]:
test_info['hour'] = round(test_info['dt'].dt.hour/2)
print(test_info)

          id      userid      docid      timestamp  network  refresh  \
0          0  2204868548  464495940  1625011774995        2        4   
1          1  1623790870  464553715  1625066054894        2        2   
2          2  1637010070  464687570  1625053546711        2       10   
3          3  1634251474  464661865  1625047014335        2       10   
4          4  2352721164  464692079  1625035383911        2       11   
...      ...         ...        ...            ...      ...      ...   
49995  49995  1640585246  464553715  1625036568698        2        1   
49996  49996  1987843480  464382231  1625009469794        2        1   
49997  49997  2391080328  464617167  1625027566512        5        1   
49998  49998   125699342  464172118  1625034692217        2       11   
49999  49999  2154437014  464225813  1625008637470        2        3   

                                    dt        date  hour  
0     2021-06-30 08:09:34.995000+08:00  2021-06-30   4.0  
1     2021-06-30 

In [19]:
# del test_info['timestamp']
# del test_info['dt']
del test_info['date']
print(test_info)

          id      userid      docid  network  refresh  hour
0          0  2204868548  464495940        2        4   4.0
1          1  1623790870  464553715        2        2  12.0
2          2  1637010070  464687570        2       10  10.0
3          3  1634251474  464661865        2       10   8.0
4          4  2352721164  464692079        2       11   7.0
...      ...         ...        ...      ...      ...   ...
49995  49995  1640585246  464553715        2        1   8.0
49996  49996  1987843480  464382231        2        1   4.0
49997  49997  2391080328  464617167        5        1   6.0
49998  49998   125699342  464172118        2       11   7.0
49999  49999  2154437014  464225813        2        3   4.0

[50000 rows x 6 columns]


In [20]:
test_info.to_pickle('../data/out5/test5.pkl')

In [2]:
import pandas as pd
train_info = pd.read_table('../data/train_info.txt',header=None)
print(train_info)

                   0          1              2  3  4   5  6    7
0         1000014754  463510256  1624843756147  5  0  16  0    0
1         1000014754  463852707  1624843756147  5  0  13  1   80
2         1000014754  463067100  1624757147178  5  0  13  0    0
3         1000014754  463625484  1624762446340  5  0  12  1  268
4         1000014754  463625484  1624762446340  5  0  12  1   57
...              ...        ...            ... .. ..  .. ..  ...
81671128   999938860  463459430  1624885221612  2  2  20  0    0
81671129   999938860  463930923  1624884995834  2  1  15  1    5
81671130   999938860  463939123  1624885221612  2  2  23  0    0
81671131   999938860  463270375  1624747290961  2  7  67  0    0
81671132   999938860  463247124  1624747290961  2  7  69  0    0

[81671133 rows x 8 columns]


In [3]:
train_info.columns = ['userid','docid','timestamp','network','refresh','position','click','duration']
print(train_info)

              userid      docid      timestamp  network  refresh  position  \
0         1000014754  463510256  1624843756147        5        0        16   
1         1000014754  463852707  1624843756147        5        0        13   
2         1000014754  463067100  1624757147178        5        0        13   
3         1000014754  463625484  1624762446340        5        0        12   
4         1000014754  463625484  1624762446340        5        0        12   
...              ...        ...            ...      ...      ...       ...   
81671128   999938860  463459430  1624885221612        2        2        20   
81671129   999938860  463930923  1624884995834        2        1        15   
81671130   999938860  463939123  1624885221612        2        2        23   
81671131   999938860  463270375  1624747290961        2        7        67   
81671132   999938860  463247124  1624747290961        2        7        69   

          click  duration  
0             0         0  
1      

In [4]:
train_info = train_info.sample(frac=0.1)
print(train_info)

              userid      docid      timestamp  network  refresh  position  \
41400510  1297726476  463328005  1624695742723        2        6        59   
58998176  1487564970  463370683  1624783737485        2       21       215   
26385549  1491755878  464151864  1624921556048        5        3        45   
18936254  1629766146  463606364  1624798166732        2       12        57   
45517337  2234732820  462690613  1624570834648        2        2        32   
...              ...        ...            ...      ...      ...       ...   
55168816  2410479102  463787002  1624845076865        2        0        16   
49534688  1276047728  463454647  1624770161552        2       26       134   
1802329   1485372394  462227794  1624564660184        5       33       186   
908883    1354374344  463898293  1624948443387        2        9        18   
27811073  1987779980  463465777  1624717807122        2       25       264   

          click  duration  
41400510      0         0  
5899817

In [5]:
train_info['dt'] = pd.to_datetime(train_info['timestamp'], utc=True, unit='ms').dt.tz_convert('Asia/Shanghai')
train_info['hour'] = round(train_info['dt'].dt.hour/2)
print('train data')
print(train_info)


train data
              userid      docid      timestamp  network  refresh  position  \
41400510  1297726476  463328005  1624695742723        2        6        59   
58998176  1487564970  463370683  1624783737485        2       21       215   
26385549  1491755878  464151864  1624921556048        5        3        45   
18936254  1629766146  463606364  1624798166732        2       12        57   
45517337  2234732820  462690613  1624570834648        2        2        32   
...              ...        ...            ...      ...      ...       ...   
55168816  2410479102  463787002  1624845076865        2        0        16   
49534688  1276047728  463454647  1624770161552        2       26       134   
1802329   1485372394  462227794  1624564660184        5       33       186   
908883    1354374344  463898293  1624948443387        2        9        18   
27811073  1987779980  463465777  1624717807122        2       25       264   

          click  duration                           

In [6]:
del train_info['timestamp']
del train_info['dt']
del train_info['position']
train_info=train_info.reset_index()
print(train_info)


            index      userid      docid  network  refresh  click  duration  \
0        41400510  1297726476  463328005        2        6      0         0   
1        58998176  1487564970  463370683        2       21      0         0   
2        26385549  1491755878  464151864        5        3      0         0   
3        18936254  1629766146  463606364        2       12      1       166   
4        45517337  2234732820  462690613        2        2      0         0   
...           ...         ...        ...      ...      ...    ...       ...   
8167108  55168816  2410479102  463787002        2        0      0         0   
8167109  49534688  1276047728  463454647        2       26      0         0   
8167110   1802329  1485372394  462227794        5       33      0         0   
8167111    908883  1354374344  463898293        2        9      0         0   
8167112  27811073  1987779980  463465777        2       25      0         0   

         hour  
0         8.0  
1         8.0  
2  

In [7]:
del train_info['index']
del train_info['duration']

In [8]:
print(train_info)

             userid      docid  network  refresh  click  hour
0        1297726476  463328005        2        6      0   8.0
1        1487564970  463370683        2       21      0   8.0
2        1491755878  464151864        5        3      0   4.0
3        1629766146  463606364        2       12      1  10.0
4        2234732820  462690613        2        2      0   2.0
...             ...        ...      ...      ...    ...   ...
8167108  2410479102  463787002        2        0      0   4.0
8167109  1276047728  463454647        2       26      0   6.0
8167110  1485372394  462227794        5       33      0   2.0
8167111  1354374344  463898293        2        9      0   7.0
8167112  1987779980  463465777        2       25      0  11.0

[8167113 rows x 6 columns]


In [9]:
train_info.to_pickle('../data/out5/train5_0.1.pkl')

In [10]:
train_info['click'].mean()

0.14465711445403046